## PR0205: Monitorización de rendimiento del servidor con Python

In [6]:
!pip install influxdb-client
!pip install psutil

In [25]:
import psutil
import time
import influxdb_client
from influxdb_client import Point
from influxdb_client.client.write_api import WriteOptions, WriteType
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0="
BUCKET = "Measurement"
HOST_ID = "servidor_01"
ENTORNO = "produccion"
INTERVALO_SEGUNDOS = 1 
ORG = "docs"

print("--- Iniciando conexión a InfluxDB ---")

client = None
write_api = None
try:
    print("\n--- Estableciendo Conexión a InfluxDB ---")
    
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org=ORG
    )

    health = client.health()
    if health.status == "pass":
        print(f"[INFO] Conexión exitosa a InfluxDB (v{health.version})")
    else:
        raise ConnectionError(f"[ERROR] Conexión fallida. Estado: {health.status}, Mensaje: {health.message}")

    write_api = client.write_api()
    print(f"[INFO] WriteAPI configurada en modo Batching (ASYNCHRONOUS).")

except (InfluxDBError, NewConnectionError) as e:
    print(f"\n[ERROR CRÍTICO] Error al conectar con InfluxDB: {e}")
    client = None 
    write_api = None


--- Iniciando conexión a InfluxDB ---

--- Estableciendo Conexión a InfluxDB ---
[INFO] Conexión exitosa a InfluxDB (vv2.7.12)
[INFO] WriteAPI configurada en modo Batching (ASYNCHRONOUS).


### Obtención métricas

In [26]:
import psutil
import time

# Obtener estadísticas de uso
def obtener_metricas_sistema(host_id, interval):
    # Uso de CPU (promedio de los últimos segundos)
    cpu_usage = psutil.cpu_percent(interval=interval)
    
    # Uso de RAM
    mem = psutil.virtual_memory()
    ram_used_gb = round(mem.used / (1024**3), 2) # Conversión a GB
    ram_percent = mem.percent
    
    # Uso de disco (en el punto de montaje raíz)
    disk = psutil.disk_usage('/')
    disk_percent = disk.percent
    
    return {
        'host': host_id,
        'cpu_percent': cpu_usage,
        'ram_used_gb': ram_used_gb,
        'ram_percent': ram_percent,
        'disk_percent': disk_percent
    }

### Modelado de métricas


In [27]:
def crear_punto_influxdb(datos_metricas, entorno):
    point = Point("rendimiento_servidor") \
        .tag("host_id", datos_metricas['host']) \
        .tag("entorno", entorno) \
        .field("cpu_percent", datos_metricas['cpu_percent']) \
        .field("ram_percent", datos_metricas['ram_percent']) \
        .field("disk_percent", datos_metricas['disk_percent'])
    
    return point

### Monitorización y escritura

In [28]:
if write_api:
    try:
        print(f"\n[INFO] Iniciando monitorización continua (cada {INTERVALO_SEGUNDOS} segundo/s).")
        print("-> Detenga la celda para finalizar la escritura y vaciar el buffer.")
        
        while True:
            metricas = obtener_metricas_sistema(HOST_ID, INTERVALO_SEGUNDOS)
            
            punto = crear_punto_influxdb(metricas, ENTORNO)
            
            write_api.write(bucket=BUCKET, org=ORG, record=punto)
            
    except KeyboardInterrupt:
        print("\n[INFO] Interrupción por el usuario (KeyboardInterrupt) detectada.")
    except Exception as e:
        print(f"\n[ERROR CRÍTICO] Ocurrió un error en el bucle: {e}")
else:
    print("[ERROR] No se puede iniciar el bucle. La WriteAPI no se inicializó correctamente.")


[INFO] Iniciando monitorización continua (cada 1 segundo/s).
-> Detenga la celda para finalizar la escritura y vaciar el buffer.

[INFO] Interrupción por el usuario (KeyboardInterrupt) detectada.


In [29]:
if write_api:
    try:
        print("[INFO] Cerrando WriteAPI y vaciando el buffer de puntos restantes...")
        write_api.close()
        print("[INFO] Buffer vaciado correctamente.")
    except Exception as e:
        print(f"[ERROR] Error al cerrar WriteAPI: {e}")

if client:
    client.close()
    print("--- Conexión a InfluxDB cerrada y recursos liberados ---")

[INFO] Cerrando WriteAPI y vaciando el buffer de puntos restantes...
[INFO] Buffer vaciado correctamente.
--- Conexión a InfluxDB cerrada y recursos liberados ---
